<a href="https://colab.research.google.com/github/bhushanmandava/LLM_fine_tuning/blob/main/health.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall accelerate peft bitsandbytes transformers trl -y
!pip install accelerate peft==0.13.2 bitsandbytes transformers trl==0.12.0

Found existing installation: accelerate 1.5.2
Uninstalling accelerate-1.5.2:
  Successfully uninstalled accelerate-1.5.2
Found existing installation: peft 0.13.2
Uninstalling peft-0.13.2:
  Successfully uninstalled peft-0.13.2
Found existing installation: bitsandbytes 0.45.4
Uninstalling bitsandbytes-0.45.4:
  Successfully uninstalled bitsandbytes-0.45.4
Found existing installation: transformers 4.50.3
Uninstalling transformers-4.50.3:
  Successfully uninstalled transformers-4.50.3
Found existing installation: trl 0.12.0
Uninstalling trl-0.12.0:
  Successfully uninstalled trl-0.12.0
  Using cached accelerate-1.5.2-py3-none-any.whl.metadata (19 kB)
  Using cached peft-0.13.2-py3-none-any.whl.metadata (13 kB)
  Using cached bitsandbytes-0.45.4-py3-none-manylinux_2_24_x86_64.whl.metadata (5.0 kB)
  Using cached transformers-4.50.3-py3-none-any.whl.metadata (39 kB)
  Using cached trl-0.12.0-py3-none-any.whl.metadata (10 kB)
Using cached peft-0.13.2-py3-none-any.whl (320 kB)
Using cached tr

In [2]:
!pip install datasets


In [3]:
import os
import torch
from trl import SFTTrainer
from datasets import load_dataset
from peft import LoraConfig, PeftModel
from transformers import (AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser ,TrainingArguments,pipeline,logging)

In [4]:
model_base ="aboonaji/llama2finetune-v2"#  llama version 2 fine tuned model
source_dataset = "gamino/wiki_medical_terms" # gettin the dataset which is  medical terms

formatted_dataset = "aboonaji/wiki_medical_terms_llam2_format" # this the data which is formmated

In [21]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [5]:
lora_hyper_r = 64
lora_hyper_alpha = 16
lora_hyper_dropout = 0.1

In [6]:
enable_4bit = True
compute_dtype_bnb = "float16"
quant_type_bnb = "nf4"
double_quant_flag = False

In [7]:
results_dir = "./results"
epochs_count = 10
enable_fp16 = False
enable_bf16 = False
train_batch_size = 4
eval_batch_size = 4
accumulation_steps = 1
checkpointing_flag = True
grad_norm_limit = 0.3
train_learning_rate = 2e-4
decay_rate = 0.001
optimizer_type = "paged_adamw_32bit"
scheduler_type = "cosine"
steps_limit = 100
warmup_percentage = 0.03
length_grouping = True
checkpoint_interval = 0
log_interval = 25

In [8]:
enable_packing = False
sequence_length_max = None
device_assignment = {"": 0}

In [9]:
training_data = load_dataset(formatted_dataset, split = "train")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [10]:
training_data

Dataset({
    features: ['text'],
    num_rows: 6861
})

In [11]:
dtype_computation = getattr(torch, compute_dtype_bnb)
bnb_setup = BitsAndBytesConfig(load_in_4bit = enable_4bit,
                               bnb_4bit_quant_type = quant_type_bnb,
                               bnb_4bit_use_double_quant = double_quant_flag,
                               bnb_4bit_compute_dtype = dtype_computation)

In [12]:
llama_model = AutoModelForCausalLM.from_pretrained(model_base, quantization_config = bnb_setup, device_map = device_assignment)
llama_model.config.use_case = False
llama_model.config.pretraining_tp = 1

config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

adapter_model.bin:   0%|          | 0.00/33.6M [00:00<?, ?B/s]

In [14]:
llama_tokenizer = AutoTokenizer.from_pretrained(model_base, trust_remote_code = True)
llama_tokenizer.pad_token = llama_tokenizer.eos_token
llama_tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [15]:
peft_setup = LoraConfig(lora_alpha = lora_hyper_alpha,
                        lora_dropout = lora_hyper_dropout,
                        r = lora_hyper_r,
                        bias = "none",
                        task_type = "CAUSAL_LM")

In [16]:
train_args = TrainingArguments(output_dir = results_dir,
                               num_train_epochs = epochs_count,
                               per_device_train_batch_size = train_batch_size,
                               per_device_eval_batch_size = eval_batch_size,
                               gradient_accumulation_steps = accumulation_steps,
                               learning_rate = train_learning_rate,
                               weight_decay = decay_rate,
                               optim = optimizer_type,
                               save_steps = checkpoint_interval,
                               logging_steps = log_interval,
                               fp16 = enable_fp16,
                               bf16 = enable_bf16,
                               max_grad_norm = grad_norm_limit,
                               max_steps = steps_limit,
                               warmup_ratio = warmup_percentage,
                               group_by_length = length_grouping,
                               lr_scheduler_type = scheduler_type,
                               gradient_checkpointing = checkpointing_flag)

In [22]:
# Check for GPU and move model to CUDA
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
llama_model.to(device)
llama_sftt_trainer = SFTTrainer(model = llama_model,
                                args = train_args,
                                train_dataset = training_data,
                                tokenizer = llama_tokenizer,
                                peft_config = peft_setup,
                                dataset_text_field = "text",
                                max_seq_length = sequence_length_max,
                                packing = enable_packing)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/6861 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [23]:

# Double-check
print(next(llama_model.parameters()).device)  # Should print 'cuda:0'


cuda:0


In [24]:
llama_sftt_trainer.train()

<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: bhushanmandava16 (bhushanmandava16-personal) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
25,1.712600
50,0.850100
75,1.392300


Step,Training Loss
25,1.712600
50,0.850100
75,1.392300
100,0.760400


TrainOutput(global_step=100, training_loss=1.1788517951965332, metrics={'train_runtime': 3365.4835, 'train_samples_per_second': 0.119, 'train_steps_per_second': 0.03, 'total_flos': 1.1667048453832704e+16, 'train_loss': 1.1788517951965332, 'epoch': 0.05827505827505827})

In [25]:
user_prompt = "Please tell me about Bursitis"
text_generation_pipe = pipeline(task = "text-generation", model = llama_model, tokenizer = llama_tokenizer, max_length = 500)
generation_result = text_generation_pipe(f"<s>[INST] {user_prompt} [/INST]")
print(generation_result[0]['generated_text'])

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gr

<s>[INST] Please tell me about Bursitis [/INST]  Bursitis is inflammation of the bursae, small fluid-filled sacs that cushion and reduce friction between bones, tendons, and ligaments. Hinweis: Bursitis can be caused by a variety of factors, including injury, repetitive motion, infection, and gout.

Symptoms
Bursitis symptoms include:
* Pain in the affected area
* Redness and warmth in the affected area
* Swelling in the affected area
* Limited mobility in the affected joint
* Stiffness in the affected joint
* Painful movement in the affected joint

Causes
Bursitis can be caused by a variety of factors, including:
* Injury: Bursitis can occur after an injury to the affected joint, such as a fall or a blow to the area.
* Repetitive motion: Repeated motion in a particular joint, such as kneeling or bending, can cause inflammation in the bursae.
* Infection: Bursitis can be caused by bacterial or viral infections, such as staphylococcus or herpes simplex virus.
* Gout: Bursitis can occur 